In [ ]:
# code for importing the necessary libraries and modules for machine learning and data analysis.

import os
import json
import pickle
import random
import sklearn
import requests
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.svm import SVC
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn import tree, metrics
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer
from flask import Flask,request, jsonify
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error


drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Loading player 21 data from a CSV file
players21_data=pd.read_csv('/content/drive/My Drive/Colab Notebooks/players_21.csv')

In [ ]:
# Loading player 22 data from a CSV file
players22_data=pd.read_csv('/content/drive/My Drive/Colab Notebooks/players_22.csv')

<ipython-input-62-6524dbd0984d>:2: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players22_data=pd.read_csv('/content/drive/My Drive/Colab Notebooks/players_22.csv')


DATA PREPROCESSING

---



In [ ]:
# Generating summary statistics for the player 21 data.
players21_data.describe()

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,18944.000000,18944.000000,18944.000000,1.870700e+04,18719.000000,18944.000000,18944.000000,18944.000000,18719.000000,18719.000000,...,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,2083.000000
mean,226232.328917,65.677787,71.086729,2.902288e+06,9148.482825,25.225823,181.190773,75.016892,47908.905551,1.355468,...,57.978674,46.470017,47.581767,45.546505,16.446052,16.236486,16.103357,16.225982,16.551309,37.203553
std,27166.574284,7.002278,6.109985,7.743775e+06,19893.149956,4.697354,6.825672,7.057140,53906.733776,0.739015,...,12.118390,20.169591,21.402461,20.953997,17.577332,16.845480,16.519399,17.017341,17.878121,10.714523
min,41.000000,47.000000,47.000000,9.000000e+03,500.000000,16.000000,155.000000,50.000000,1.000000,1.000000,...,12.000000,3.000000,5.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,12.000000
25%,210028.750000,61.000000,67.000000,4.750000e+05,1000.000000,21.000000,176.000000,70.000000,461.000000,1.000000,...,50.000000,29.000000,27.000000,24.000000,8.000000,8.000000,8.000000,8.000000,8.000000,28.000000
50%,232301.500000,66.000000,71.000000,9.750000e+05,3000.000000,25.000000,181.000000,75.000000,1913.000000,1.000000,...,59.000000,52.000000,55.000000,52.000000,11.000000,11.000000,11.000000,11.000000,11.000000,37.000000
75%,246745.750000,70.000000,75.000000,2.000000e+06,8000.000000,29.000000,186.000000,80.000000,110986.000000,1.000000,...,66.000000,63.000000,65.000000,63.000000,14.000000,14.000000,14.000000,14.000000,14.000000,45.000000
max,258970.000000,93.000000,95.000000,1.855000e+08,560000.000000,53.000000,206.000000,110.000000,114899.000000,4.000000,...,96.000000,94.000000,93.000000,90.000000,90.000000,92.000000,93.000000,91.000000,90.000000,65.000000


In [ ]:
# Displaying the first few rows of player data.
players21_data.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,103500000.0,560000.0,33,...,52+3,52+3,52+3,62+3,19+3,https://cdn.sofifa.net/players/158/023/21_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,92,63000000.0,220000.0,35,...,54+3,54+3,54+3,61+3,20+3,https://cdn.sofifa.net/players/020/801/21_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
2,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,91,91,111000000.0,240000.0,31,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/21_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,NaN,https://cdn.sofifa.net/flags/pl.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,132000000.0,270000.0,28,...,49+3,49+3,49+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/21_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CAM, CM",91,91,129000000.0,370000.0,29,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/21_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


In [ ]:
# Removing specific columns from the 'players21_data' DataFrame as they are derived from existing features and have no bearing on the rating.
players21_data=players21_data.drop(['sofifa_id','ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram','lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb','lb', 'lcb', 'cb', 'rcb', 'rb', 'gk','player_url','player_face_url','player_positions','club_logo_url','club_flag_url','club_joined','dob','nation_flag_url','short_name', 'long_name'],axis=1)

In [ ]:
# Removing specific columns from the 'players22_data' DataFrame as they are derived from existing features and have no bearing on the rating.
players22_data=players22_data.drop(['sofifa_id','ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram','lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb','lb', 'lcb', 'cb', 'rcb', 'rb', 'gk','player_url','player_face_url','player_positions','club_logo_url','club_flag_url','club_joined','dob','nation_flag_url','short_name', 'long_name'],axis=1)

In [ ]:
# Dropping columns that have 30% or more of their values as null to reduce bias
biased_values =  0.30 * len(players21_data)
players21_data = players21_data.loc[:, players21_data.isna().sum() < biased_values]

In [ ]:
# Dropping columns from 'players22_data' that are not present in 'players21_data'.
dropped_columns = []
for column_name in players22_data.columns:
    if column_name not in players21_data.columns:
        dropped_columns.append(column_name)

players22_data=players22_data.drop(dropped_columns,axis=1)

In [ ]:
# Handling missing values by filling numerical columns with their mean.
numerical_columns = ["overall", "potential", "value_eur", "wage_eur", "age", "height_cm", "weight_kg", "club_team_id", "league_level", "club_jersey_number", "club_contract_valid_until", "nationality_id", "weak_foot", "skill_moves", "international_reputation", "release_clause_eur", "pace", "shooting", "passing", "dribbling", "defending", "physic", "attacking_crossing", "attacking_finishing", "attacking_heading_accuracy", "attacking_short_passing", "attacking_volleys", "skill_dribbling", "skill_curve", "skill_fk_accuracy", "skill_long_passing", "skill_ball_control", "movement_acceleration", "movement_sprint_speed"
,"movement_agility", "movement_reactions", "movement_balance", "power_shot_power", "power_jumping", "power_stamina", "power_strength", "power_long_shots", "mentality_aggression", "mentality_interceptions", "mentality_positioning", "mentality_vision", "mentality_penalties", "mentality_composure", "defending_marking_awareness", "defending_standing_tackle", "defending_sliding_tackle", "goalkeeping_diving", "goalkeeping_handling", "goalkeeping_kicking", "goalkeeping_positioning", "goalkeeping_reflexes"]
categorical_columns=['club_name','league_name','real_face','club_position','nationality_name','preferred_foot','work_rate','body_type']
players21_data[numerical_columns] = players21_data[numerical_columns].apply(lambda col: col.fillna(np.mean(col)))
players22_data[numerical_columns] = players22_data[numerical_columns].apply(lambda col: col.fillna(np.mean(col)))

In [ ]:
# Impute missing values with mode for players21_data
for column in categorical_columns:
    mode = players21_data[column].mode()[0]
    players21_data[column].fillna(mode, inplace=True)

# Impute missing values with mode for players22_data
for column in categorical_columns:
    mode = players22_data[column].mode()[0]
    players22_data[column].fillna(mode, inplace=True)

In [ ]:
# Filling missing values in 'players22_data' numerical columns with their means.
players22_data[numerical_columns] = players22_data[numerical_columns].apply(lambda col: col.fillna(np.mean(col)))

FEATURE ENGINEERING

---



In [ ]:
 # One-hot encoding categorical columns in 'players21_data'.
one_hot_encoded = pd.get_dummies(players21_data[categorical_columns], columns=categorical_columns)
players21_data = players21_data.drop(columns=categorical_columns)
players21_data = pd.concat([players21_data, one_hot_encoded], axis=1)

In [ ]:
# One-hot encoding categorical columns in 'players22_data'.
one_hot_encoded_2 = pd.get_dummies(players22_data[categorical_columns], columns=categorical_columns)
players22_data = players22_data.drop(columns=categorical_columns)
players22_data = pd.concat([players22_data, one_hot_encoded_2], axis=1)

In [ ]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(players21_data.drop('overall', axis=1), players21_data['overall'], test_size=0.2, random_state=0)

# Create a Random Forest Regressor
forest = RandomForestRegressor(n_estimators=250, random_state=0)

# Fit the model on the training data
forest.fit(X_train, y_train)


RandomForestRegressor(n_estimators=250, random_state=0)

In [ ]:
# Get feature importances
feature_importances = forest.feature_importances_

# Get feature names
feature_names = X_train.columns

# Create a dictionary of feature names and their importances
feature_importance_dict = dict(zip(feature_names, feature_importances))

# Sort features by importance (from highest to lowest)
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# Select the top 13 features based on importance
selected_features = sorted_features[:18]

# Extract the names of the selected features
selected_feature_names = [feature[0] for feature in selected_features]


In [ ]:
# Dropping of columns that are categorical and encoded
selected_feature_names.remove('nationality_name_New Zealand')
selected_feature_names.remove('league_name_Campeonato Brasileiro Série A')
selected_feature_names.remove('physic')


In [ ]:
players21_data[selected_feature_names]

,value_eur,release_clause_eur,age,potential,movement_reactions,defending,wage_eur,mentality_interceptions,attacking_crossing,goalkeeping_diving,goalkeeping_reflexes,defending_marking_awareness,goalkeeping_positioning,mentality_composure,mentality_penalties
0,103500000.0,138400000.0,33,93,94,38.0,560000.0,40,85,6,8,32,14,96,75
1,63000000.0,75900000.0,35,92,95,35.0,220000.0,29,84,7,11,28,14,95,84
2,111000000.0,132000000.0,31,91,93,43.0,240000.0,49,71,15,10,35,8,88,88
3,132000000.0,166500000.0,28,91,91,36.0,270000.0,36,85,9,11,35,15,93,92
4,129000000.0,161000000.0,29,91,91,64.0,370000.0,66,94,15,13,68,10,91,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,70000.0,57000.0,21,52,48,50.0,1000.0,50,23,12,6,45,14,40,35
18940,70000.0,72000.0,21,53,50,43.0,1000.0,42,38,8,10,43,14,35,35
18941,45000.0,47000.0,28,47,44,40.0,2000.0,35,44,8,7,38,5,35,36
18942,130000.0,165000.0,17,67,53,16.0,500.0,11,27,11,6,18,9,45,50


In [ ]:
# Selecting specific features for 'X' and 'X1'.
X = players21_data[selected_feature_names]
X_22 = players22_data[selected_feature_names]

In [ ]:
# Scaling feature data for 'X' and 'X1', and creating new DataFrames.
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

scaled_players21_data = pd.concat([X_scaled, players21_data['overall']], axis=1)

X_22_scaled = pd.DataFrame(scaler.fit_transform(X_22), columns=X_22.columns)
scaled_players22_data = pd.concat([X_22_scaled, players22_data['overall']], axis=1)

TRAINING

---



In [ ]:
# Assigning scaled feature data and splitting it into training and testing sets.
X=X_scaled
y=scaled_players21_data['overall']

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2,  random_state=42)

RANDOM FOREST

In [ ]:
# Training a Random Forest Regressor model and making predictions.
rf=RandomForestRegressor()
rf.fit(X_train, Y_train)
y_pred_rf =rf.predict(X_test)
print(y_pred_rf)

[59.95 64.04 58.   ... 72.84 62.98 61.42]


In [ ]:
# Testing the Random Forest Regressor
mae_rf = mean_absolute_error(Y_test, y_pred_rf)
print(mae_rf)

0.2482818685669044


XGBOOST
---




In [ ]:
# Create a DMatrix, train an XGBoost Regressor, make predictions, and return the predictions.
data_dmatrix = xgb.DMatrix(data=X_train,label=Y_train)
xg_reg = xgb.XGBRegressor()
xg_reg.fit(X_train,Y_train)
y_pred_xg_reg = xg_reg.predict(X_test)
y_pred_xg_reg

array([59.78853 , 63.88067 , 57.821205, ..., 72.92589 , 63.05091 ,
       61.669384], dtype=float32)

In [ ]:
# Calculate and return the Mean Absolute Error (MAE) for XGBoost Regressor predictions.
mae_xg_reg = mean_absolute_error(Y_test, y_pred_xg_reg)
mae_xg_reg

0.33205817638647905

GRADIENT BOOST

In [ ]:
# Train a Gradient Boosting Regressor, make predictions, and return the predictions.
gbr = GradientBoostingRegressor()
gbr.fit(X_train, Y_train)
y_pred_gbr= gbr.predict(X_test)
y_pred_gbr

array([59.81698864, 63.31263788, 57.93890382, ..., 73.04665619,
       63.18738135, 61.02519052])

In [ ]:
# Calculate and return the Mean Absolute Error (MAE) for Gradient Boosting Regressor predictions.
mae_gbr = mean_absolute_error(Y_test, y_pred_gbr)
mae_gbr

0.5722140744219825

CROSS-VALIDATION AND FINETUNING

---



In [ ]:
# Setup 3-fold cross-validation, define hyperparameter grid, perform grid search, and fit the model.
#Since the RandomForestRegressor had the lowest MAE, we decided to use it for finetuning.
cv=KFold(n_splits=3)
PARAMETERS ={
'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, 40, 50]
}

model_rf = GridSearchCV(rf, param_grid=PARAMETERS,cv=cv,scoring='neg_mean_absolute_error')
model_rf.fit(X_train,Y_train)

GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
             estimator=RandomForestRegressor(),
             param_grid={'max_depth': [10, 20, 30, 40, 50],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_absolute_error')

In [ ]:
# Find best hyperparameters, train the best RandomForest Regressor, make predictions, and calculate MAE.
best_params=model_rf.best_params_
best_rf_regressor = RandomForestRegressor(**best_params)
best_rf_regressor.fit(X_train, Y_train)
y_pred_best_rf_regressor = best_rf_regressor.predict(X_test)
mae_best_rf_regressor = mean_absolute_error(Y_test, y_pred_best_rf_regressor)
mae_best_rf_regressor


0.24863904284331814

TESTING WITH PLAYERS DATA FROM 2022

---



In [ ]:
# Assign variables X1 and y1 for feature data and target data, respectively.
X_22=X_22_scaled
y_22=scaled_players22_data['overall']

In [ ]:
# Split data into training and testing sets for features (X) and target (Y).
X_train=X_scaled
X_test=X_22_scaled
Y_train=y
Y_test=y_22

In [ ]:
# Train a RandomForest Regressor with the best hyperparameters, make predictions, and return the predictions.
rf_22=RandomForestRegressor(**best_params)
rf_22.fit(X_train, Y_train)
y_pred_rf_22 =rf.predict(X_test)
y_pred_rf_22

array([89.99, 90.6 , 89.8 , ..., 52.77, 53.05, 53.  ])

In [ ]:
# Calculate and return the Mean Absolute Error (MAE) for RandomForest Regressor predictions on a new dataset.
mae_rf_22 = mean_absolute_error(Y_test, y_pred_rf_22)
mae_rf_22

0.5564197723374397

DEPLOYMENT

---



---



In [ ]:
# Save the trained RandomForest Regressor model to a pickle file.
with open('/content/drive/My Drive/Colab Notebooks/model.pkl', 'wb') as file:
    pickle.dump(best_rf_regressor, file)

In [ ]:
# Find the average of the overall feature and use that as a true value/target value for finding the confidence level
scaled_players21_data['overall'].mean()


65.67778716216216